Let's solve the scalar Newell–Whitehead-Segel reaction-diffusion equation on $(t,x)\in[0,\infty)\times[-1,1]$,

$$\partial_t q(t,x) - \nu(x) \partial_{xx} q(t,x) - \partial_x\nu(x)\partial_x q(t,x) - q(t,x) (1-q^2(t,x)) = 0.$$



In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

In [ ]:
def dct_chebyshev_transform(f):
    N = len(f)
    coefs = f
    coefs = sp.fft.dct(coefs,orthogonalize=True,norm='ortho')/np.sqrt(0.5*N)
    coefs[0] = coefs[0]/np.sqrt(2)
    return coefs

def idct_chebyshev_transform(f):
    N = len(f)
    icoefs = np.sqrt(N/2)*f
    icoefs[0] = np.sqrt(2)*icoefs[0]
    icoefs = sp.fft.idct(icoefs,orthogonalize=True,norm='ortho')
    return icoefs

def get_collocation_points(x0,x1,N):
    #Map to Chebyshev domain
    x = np.pi*(np.arange(N)+0.5)/np.float64(N)
    x = 0.5*(x1+x0)-0.5*(x1-x0)*np.cos(x)
    return x

def multiplication_coef(s,j,k,lam):
    t1 = j + k + lam
    t2 = t1 - s
    t1 = t1 - 2*s
    
    #First term
    cs = t1/t2
    
    t2 = 1
    for t in range(s - 1):
        t2 = t2*(lam + t)/(1 + t)
    
    #Second term
    cs = cs*t2
    
    t3 = 1 
    for t in range(j - s - 1):
        t3 = t3*(lam + t)/(1 + t)
    
    #Third term
    cs = cs*t3
    
    t4 = 1
    for t in range(s - 1):
        t1 = j + k - 2*s + t
        t2 = t1 + lam
        t1 = t1 + 2*lam
        t1 = t1/t2
        t4 = t4*t1
    
    #Fourth term
    cs = cs*t4
    
    t5 = 1
    for t in range(j-s-1):
        t1 = k - s + t
        t2 = t1 + lam
        t1 = t1 + 1
        t1 = t1/t2
        t5 = t5*t1
        
    #Fifth term
    cs = cs*t5
    return cs

def build_multiplication_matrix(coefs,N,lam):
    Mlam = sparse.lil_matrix((N,N),dtype=np.float64)
    ncoef = len(coefs)
    for j in range(N):
        for k in range(N):
            t = 0
            i1 = int(np.max([0,k-j]))
            inds = np.array(2*np.arange(i1,k,1)+j-k).astype(int)
            ainds = np.where((inds>=0)&(inds<ncoef))[0]
            sinds = np.arange(i1,k,1,dtype=int)
            for a in ainds:
                cs = multiplication_coef(sinds[a],k,inds[a],lam)
                t = t + cs*coefs[inds[a]]
            Mlam[j,k] = t
    Mlam.tocsc()
    return Mlam

def build_derivative_matrix(N,lam):
    elements = 2**(lam-1)*np.math.factorial(lam-1)*(np.arange(N,dtype=np.float64)+lam)
    Dlam = sparse.diags(elements,lam,shape=(N,N),format='csc')
    return Dlam
    
def build_conversion_matrix(N,lam):
    if (lam==0):
        upper = -np.ones(N-2)*0.5
        diag = np.ones(N)*0.5
        diag[0] = 1
        Slam = sparse.diags([diag,upper],[0,2],format='csc')
    else:
        ran = np.arange(N-2)+lam+2
        upper = -lam*np.ones(N-2)/ran
        ran = np.arange(N)+lam
        diag = lam*np.ones(N)/ran
        diag[0] = 1
        Slam = sparse.diags([diag,upper],[0,2],format='csc')   
    return Slam

def build_boundary_matrix(N):
    #Dirichlet conditions
    #x=-1
    xm1 = np.zeros(N,dtype=int)
    xp1 = np.ones(N,dtype=int)
    for n in range(N):
        xm1[n] = 1 - 2*(n % 2)
    xm1.astype(np.float64)
    Blam = sparse.lil_matrix((N,N),dtype=np.float64)
    Blam[N-2,:] = xm1
    Blam[N-1,:] = xp1
    return Blam.tocsc()
    
def build_permutation_matrix(N):
    Plam = sparse.lil_matrix((N,N),dtype=np.float64)
    for ii in range(N-2):
        Plam[ii+2,ii] = 1
    Plam[0,N-1] = 1
    Plam[1,N-2] = 1
    return Plam.tocsc()

def build_preconditioner(N):
    ran = np.arange(N-2)+2
    elements = 0.5*np.ones(N,dtype=np.float64)
    elements[0:N-2] = 0.5/ran
    Pr = sparse.diags(elements,0,shape=(N,N),dtype=np.float64,format='csc')
    return Pr

#Time parameters
ntimes = 1000000
deltat = 2.5e-6
times = np.zeros(ntimes)

#Diffusivity
nu = 0.1

#Truncation size
N = 512
Ndealias = int(N*3/2)

#Chebyshev collocation points (roots), excludes boundaries
xi = -np.cos(np.pi*(np.arange(N)+0.5)/N)

#Initial condition in physical space, and satisfying the boundary conditions.
q = 0.2*np.sin(2*np.pi*xi)**4

#Chebyshev transform
qcoefs = np.zeros(Ndealias,dtype=np.float64)
qcoefs[0:N] = dct_chebyshev_transform(q)

#Decompose the problem in spectral space
L = build_derivative_matrix(Ndealias,2)
S1 = build_conversion_matrix(Ndealias,1)
S0 = build_conversion_matrix(Ndealias,0)
B = build_boundary_matrix(Ndealias)
Pr = build_preconditioner(Ndealias)

#Mass matrix
M = S1@S0

#Linear operator
ncc_bool = False
if (ncc_bool):
    #First order term
    #dnu(x)/dx = nu * df(x)/dx = nu*4*x
    D1 = build_derivative_matrix(Ndealias,1)
    ncc = 4*xi
    ncc_coefs = dct_chebyshev_transform(ncc)
    ncc_coefs[np.where(np.abs(ncc_coefs)<1e-10)] = 0
    nncc = np.max(np.where(np.abs(ncc_coefs))[0])+1
    ncc_coefs = ncc_coefs[0:nncc]
    S0m = build_conversion_matrix(nncc,2)
    S1m = build_conversion_matrix(nncc,2)
    #Convert to C^2(x) basis from T_n(x), truncated to have as few coefs as possible.
    C = S1m@S0m
    ncc_coefs = C.dot(ncc_coefs)
    NC1 = build_multiplication_matrix(ncc_coefs,Ndealias,1)
    
    #Second order term
    #nu(x) = nu * f(x) = nu * (2*x^2 + 1)
    ncc = 2*xi**2 + 1
    ncc_coefs = dct_chebyshev_transform(ncc)
    ncc_coefs[np.where(np.abs(ncc_coefs)<1e-10)] = 0
    nncc = np.max(np.where(np.abs(ncc_coefs))[0])+1
    ncc_coefs = ncc_coefs[0:nncc]
    S0m = build_conversion_matrix(nncc,2)
    S1m = build_conversion_matrix(nncc,2)
    #Convert to C^2(x) basis from T_n(x), truncated to have as few coefs as possible.
    C = S1m@S0m
    ncc_coefs = C.dot(ncc_coefs)
    NC2 = build_multiplication_matrix(ncc_coefs,Ndealias,2)
    
    L = nu*(NC2@L + S1@NC1@D1) + M
else:
    L = nu*L + M

def build_problem_matrices(dt):
    #Define the problem matrices
    Am = M - 9/16*dt*L
    Ap = M + 3/8*dt*L
    
    #Omit the two highest order rows of A, which are replaced with the boundary operators
    Am[Ndealias-2:Ndealias-1,:] = 0
    Am = Am+B

    #Precondition the matrices
    Am = Pr@Am
    Ap = Pr@Ap
    
    #LU Decompose Am
    AmLU = sparse.linalg.splu(Am,permc_spec='COLAMD')

    return Ap, AmLU

#Initial build
Ap, AmLU = build_problem_matrices(deltat)

qkm1 = idct_chebyshev_transform(qcoefs)
qkm1s = qcoefs
qk = qcoefs
otime = 1000
numer = np.zeros((N,int(ntimes/otime)),dtype=np.float64)

time = 0e0
times[0] = 0e0
for n in range(ntimes):
    #Output
    if (n%otime==0):
        numer[:,int(n/otime)] = idct_chebyshev_transform(qk[0:N])
        
    #RHS step, MCNAB2 -- Ascher 1995
    qprev = qk
    tmp = idct_chebyshev_transform(qk)
    qp = 0.5*deltat*(3*tmp**3-qkm1**3)
    qk = Pr.dot(M@dct_chebyshev_transform(qp) + 1/16*deltat*L.dot(qkm1s)) + Ap.dot(qk)
    qkm1s = qprev
    
    #Dealias nonlinear RHS, Enforce boundary conditions
    qk[N:Ndealias] = 0

    qk = AmLU.solve(qk)
    
    time = time + deltat
    times[n] = time
    #Check dt
    #Magnitude of change in q (inf norm)
    #deltaq = np.max(np.abs(tmp-qkm1))
    #if (deltaq>0.01):
    #    deltat = 0.5*deltat
    #    Ap, AmLU = build_problem_matrices(deltat)
    #    print(deltaq, deltat)
        
    #if (deltaq<1e-6):
    #    deltat = 2*deltat
    #    Ap, AmLU = build_problem_matrices(deltat)
    #    print(deltaq, deltat)

    qkm1 = tmp
    
fig, ax = plt.subplots()
line, = ax.plot([],color='r')
time_template = 't = %.3f'
time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes)
ax.set_xlim(-1,1)
ax.set_ylim(0,np.max(numer))

def animate(frame_num):
    y = numer[:,frame_num]
    line.set_data((xi,numer[:,frame_num]))
    time_text.set_text(time_template % (times[otime*frame_num]))
    return line, time_text

anim = FuncAnimation(fig,animate,frames=int(ntimes/otime),interval=50)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()                   # avoid plotting a static plot